# Build Pseudo-nitzschia genome-scale metabolic model

In [1]:
import re

import cobra
from cobra import Reaction, Metabolite


def extract_chemical_elements(formula: str) -> dict:
    """
    Extract the chemical components from a chemical formula.
    """
    components = re.findall(r'([A-Z][a-z]*)(\d*)', formula)
    component_dict = {}
    for element, count in components:
        component_dict[element] = int(count) if count else 1
    return component_dict

def open_inorganic_exchanges(
        model: cobra.Model,
        lower_bound: float = -1000,
        upper_bound: float = 1000,
        include: list = None,
        inplace: bool = True) -> cobra.Model:
    """
    Open all inorganic exchanges in a model and close organic exchanges,
    except for indicated ones.

    Parameters
    ----------
    model : cobra.Model
        Model to open exchanges in.
    include: list, optional
        List of exchange IDs to be opened besides inorganic ones. Default is None.
    inplace : bool, optional
        If True, open exchanges in place. If False, return a copy of the model with opened
        exchanges. Default is True.

    Returns
    -------
    cobra.Model
        Model with opened exchanges.
    """
    if inplace:
        model = model
    else:
        model = model.copy()
    for rxn in model.exchanges:
        if rxn.id in include:
            rxn.lower_bound = lower_bound
            rxn.upper_bound = upper_bound
        else:
            met = [met for met in rxn.metabolites][0]
            if met.formula is not None:
                chemical_elements = extract_chemical_elements(met.formula)
                if ("C" not in chemical_elements):
                    rxn.lower_bound = lower_bound
                    rxn.upper_bound = upper_bound
                else:
                    rxn.lower_bound = 0
    return model

In [2]:
model = cobra.io.read_sbml_model("gems/photoeukstein/TARA_ARC_108_MAG_00212.cds_noSK.xml")
model

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-05


Name,TARA_ARC_108_MAG_00212_cds
Memory address,7f022551e690
Number of metabolites,2173
Number of reactions,4689
Number of genes,1931
Number of groups,357
Objective expression,1.0*bof_c - 1.0*bof_c_reverse_660d5
Compartments,"c, u, e, h"


## Add cobalamin exchange

In [3]:
# Define new metabolites
cbl1_e = Metabolite(
    'cbl1_e',
    name='Cobalamin',
    compartment='e'
    )

cbl1_c = Metabolite(
    'cbl1_c',
     name='Cobalamin',
     compartment='c'
     )

adocbl_e = Metabolite(
    'adocbl_e',
    name='Adenosylcobalamin',
    compartment='e'
    )

# Define a new exchange reaction
adocbl_ex = Reaction(
    'EX_adocbl_e',
    name='AdenosylCobalamin exchange',
    subsystem='Exchange',
    lower_bound=-1000.0,
    upper_bound=1000.0,
    )

adocbl_ex.add_metabolites({
    adocbl_e: -1.0,
})

adocbl_ex.gene_reaction_rule = 'spontaneous'

# Define transport to cytoplasm
cbl1_transport = Reaction(
    'R_cbl1_transport',
    name='Cobalamin transport',
    subsystem='Transport',
    lower_bound=0.0,
    upper_bound=1000.0,
    )

cbl1_transport.add_metabolites({
    cbl1_e: -1.0,
    cbl1_c: 1.0,
})

cbl1_transport.gene_reaction_rule = 'spontaneous'

# Define cobalamin interconversion reaction
adocbl_inter = Reaction(
    'R_cbl_interconversion_e',
    name='Cobalamin interconversion',
    subsystem='Transport',
    lower_bound=-1000.0,
    upper_bound=1000.0
    )
adocbl_inter.name = 'Cobalamin interconversion'
adocbl_inter.subsystem = 'Transport'

adocbl_inter.add_metabolites({
    cbl1_e: -1.0,
    adocbl_e: 1.0,
})

adocbl_inter.gene_reaction_rule = 'spontaneous'

# Add the reactions to the model
model.add_reactions([adocbl_ex, cbl1_transport, adocbl_inter])

## Add vitamin component to biomass reaction

Following the biomass reaction of the model iTPS1432 of the diatom _Thalassiosira pseudonana_ (https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0241960), we will add a vitamin component to the biomass reaction of Pseudo-nitzschia to guarantee that a set of essential cofactors are required for growth.

We will also need to add the fake metabolite `biomass_vit_c` to the model, as well as to add the vitamin fake metabolite to the biomass reaction of the Pseudo-nitzschia model (bof_c). Let's go ahead and modify the cobra model object.


### Add new fake metabolite biomass_vit_c

In [4]:
biomass_vit_c = Metabolite(
    "biomass_vit_c",
    formula="",
    name="vitamin component of biomass",
    compartment="c"
)
model.add_metabolites([biomass_vit_c])


## Add vitamin component reaction and modify biomass reaction

In [5]:
# Add reaction for vitamin C component of biomass
reaction_biomass_vit_c = Reaction(
    "biomass_vit_c",
    name="vitamin component of biomass",
    subsystem="",
    lower_bound=0,
    upper_bound=1000
)

reaction_biomass_vit_c.add_metabolites({
    model.metabolites.get_by_id("btn_c"): -0.00180467621671466,
    model.metabolites.get_by_id("pnto__R_c"): -0.0312336793120744,
    model.metabolites.get_by_id("thm_c"): -0.158396573308146,
    model.metabolites.get_by_id("pydxn_c"): -0.011841652061051,
    model.metabolites.get_by_id("cbl1_c"): -4.72255301000285e-06,
    model.metabolites.get_by_id("ribflv_c"): -0.0764535255651577,
    model.metabolites.get_by_id("fol_c"): -0.0573780622072159,
    model.metabolites.get_by_id("ascb__L_c"): -3.25323755030631,
    model.metabolites.get_by_id("amet_c"): -0.000163843675857242,
    model.metabolites.get_by_id("avite1_c"): -0.71543826528432,
    model.metabolites.get_by_id("biomass_vit_c"): 1,
})

model.add_reactions([reaction_biomass_vit_c])

# Modify biomass reaction
bof_c = model.reactions.get_by_id("bof_c")
bof_c.add_metabolites({
    model.metabolites.get_by_id("biomass_vit_c"): -1,
})

## Open only inorganic exchanges

Next, we are going to open only the inorganic exchanges of the model, with the exception of cofactors cobalamin, folate and thiamin, since Pseudo-nitzschia is auxotrophic for all of them. We include Co2 and HCO3 as inorganic carbon sources.

In [6]:
exchanges_to_include = [
    "EX_adocbl_e",
    "EX_hco3_e",
    "EX_co2_e",
    "EX_thm_e",
    "EX_fol_e",
    ]

model = open_inorganic_exchanges(model, lower_bound=-1000, include=exchanges_to_include)

In [7]:
model.optimize()

,fluxes,reduced_costs
ATPCS_c,0.000000,0.000000e+00
CISY_m,0.000000,0.000000e+00
PDH_E1a_h,0.000000,-3.455683e-02
ADH_c,0.000000,0.000000e+00
GAPDH_h,248.985383,0.000000e+00
...,...,...
AROH,0.530420,0.000000e+00
EX_adocbl_e,-0.000007,-0.000000e+00
R_cbl1_transport,0.000007,0.000000e+00
R_cbl_interconversion_e,-0.000007,-0.000000e+00


## Write modified model to file

In [8]:
cobra.io.write_sbml_model(model, "gems/MAG_00212_pseudonitzschia_photoeuk_noSK.xml")